### Notebook para analizar los resultados obtenidos de la simulación de Ising

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import csv

In [ ]:
global T_range
T_step = 0.5
T_range = np.arange(4.0, 0, -T_step)

## Magnetización a lo largo de la evolución

In [ ]:
for T in  T_range:
    df = pd.read_csv(f'./Resultados/rm0_50_{T}_20_100_10.csv', header= None)
    data = df.to_numpy()[0]
    M = len(data)
    
    fig, ax = plt.subplots()
    ax.set_title(f"T = {T}")
    ax.set_ylim(-1, 1)
    ax.scatter(range(M), data, marker='.')


## Estados finales del sistema

In [ ]:
for T in  T_range:
    df = pd.read_csv(f'./Resultados/sfinal_50_{T}_20_100_10.csv', header= None)
    data = df.to_numpy()[0]
    N = len(data)
    L = int(np.sqrt(N))
    
    
    fig, ax = plt.subplots()
    ax.set_title(f"T = {T}")
    ax.imshow(np.reshape(data, (L,L)), cmap='binary_r')
    # cmap = 'binary_r': negro = +1, blanco = -1
    

## Medidas generales

In [65]:
df = pd.read_csv(f'./Resultados/medidas.csv')
df


,T,L,M,M0,mc,rm,rm2,error,mc*tau,c,time_of_save
0,4.0,50,100,20,10,0.032968,6.381222e-04,0.002123,-1.467276,-0.171959,2024-03-16 20:02:41.954959
1,3.5,50,100,20,10,0.034400,6.114176e-04,0.002290,-0.710424,-0.076475,2024-03-16 20:03:03.988991
2,3.0,50,100,20,10,0.054720,2.037363e-03,0.004930,0.964393,0.087957,2024-03-16 20:03:22.498930
3,2.5,50,100,20,10,0.141096,6.799458e-03,0.021811,29.983264,0.749895,2024-03-16 20:03:51.865712
4,2.0,50,100,20,10,0.899392,2.576603e-03,0.004977,-0.193496,-0.019731,2024-03-16 20:04:12.972032
5,1.5,50,100,20,10,0.986712,1.643386e-05,NaN,-13.619154,3.763076,2024-03-16 20:04:36.760663
6,1.0,50,100,20,10,0.999296,4.899840e-07,NaN,-9.798420,-48.608150,2024-03-16 20:04:52.088750
7,0.5,50,100,20,10,1.000000,0.000000e+00,NaN,NaN,NaN,2024-03-16 20:05:06.668279
